In [ ]:
#Credit: https://www.kaggle.com/code/ngyptr/multi-class-classification-with-lstm/notebook

In [1]:
!pip uninstall -y keras
!pip uninstall -y tensorflow
!pip install --upgrade keras==2.10.0
!pip install --upgrade tensorflow==2.10.0
!pip install unidecode

Found existing installation: keras 2.12.0
Uninstalling keras-2.12.0:
  Successfully uninstalled keras-2.12.0
Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 93.0 MB/s eta 0:00:00
  Attemptin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.6 MB/s eta 0:00:00


In [3]:
!pip install keras_self_attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for keras_self_attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=6a3643f18ac3690e22e145b6784e547334c6c5ea761bd0b4784caa64148a9122
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras_self_attention


In [4]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Attention
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras_self_attention import SeqWeightedAttention

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.metrics import confusion_matrix

import re
from unidecode import unidecode
from datetime import datetime as dt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [5]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    #strategy = tf.distribute.experimental.TPUStrategy(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.6.230.106:8470
REPLICAS:  8


In [6]:
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [7]:
#df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/reGroundTruthBERT.csv', dtype='str')
#df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/CGT_ReGTR_BERT_Input.csv', dtype='str')
df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/data/GrounTruthBERT.csv', dtype='str', usecols=['TID', 'OrigTweet', 'InReplyTo', 'Target', 'Label'])
#df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/data/GrounTruthBERT.csv', dtype='str', usecols=['TID', 'CleanTweetNoHttp', 'InReplyTo', 'Target', 'Label'])
df.head()

,TID,OrigTweet,InReplyTo,Target,Label
0,826262311560216578,#coup has started. First of many steps. #rebel...,NaN,10,Politics
1,1193437298303438858,@MarkSZaidEsq @jody_prichard Funny you want to...,826262311560216578,10,Politics
2,1194280882540036098,"@MarkSZaidEsq at THAT time, the only ""stepping...",826262311560216578,10,Politics
3,1194635634960478208,@MarkSZaidEsq It's not a coup. It's a Constit...,826262311560216578,10,Politics
4,1194636052096413696,"@MarkSZaidEsq BTW, I've read your client HOSTE...",826262311560216578,10,Politics


In [8]:
df = df[df.Target.notna()]

In [9]:
len(df)

124977

In [10]:
# convert some columns to int
#df['CountReplyTweets'] = df['CountReplyTweets'].astype(float)
#df['CountReplyWords'] = df['CountReplyWords'].astype(float)
#df['CountReplyChars'] = df['CountReplyChars'].astype(float)
df['Target'] = df['Target'].astype(int)

In [11]:
def remove1stMent(text):
  if text[0] == '@':
      words = text.split()
      words = words[1:] # remove the @mention word
      text = " ".join(words).lstrip()
  return(text)

def removeSpecialChar(text):
  #old text = re.sub(r'\W', ' ', text) #replace ALL non-word characters, including emojis with space
  # remove all non ASCII characters 

  # credit: https://stackoverflow.com/questions/2758921/regular-expression-that-finds-and-replaces-non-ascii-characters-with-python
  text = re.sub(r"[\u0080-\uFFFF]", " ", text) #see ASCII list: https://www.asciitable.com/
  text = " ".join(text.split()) # replace multiple spaces with a single space
  return(text)

def removeDigits(text):
  text = re.sub(r'\d', ' ', text) #replace digits with space
  return(text)

def cleanColloquials(text):
  #replace this strange chars in the text with space
  text = text.replace("GCO"," ")
  text = text.replace("GY=n+"," ")
  text = text.replace("GCY"," ")
  text = text.replace("fcafc+"," ")
  text = text.replace("fAE+"," ")
  text = text.replace("#fAE"," ")
  text = text.replace("fnae"," ")
  text = text.replace("fye"," ")

  #Replace common abbreviations and slangs
  text = text.replace(" i m "," i am ")
  text = text.replace(" i ve "," i have ")
  text = text.replace(" i ll "," i will ")
  text = text.replace(" i d "," i had ")
  text = text.replace(" that s "," that is ")
  text = text.replace(" isn t "," is not ")
  text = text.replace(" it s "," it is ")
  text = text.replace(" she s "," she is ")
  text = text.replace(" he s "," he is ")
  text = text.replace(" u "," you ")
  text = text.replace(" ur "," your ")
  text = text.replace(" b4 "," before ")
  text = text.replace(" wasnt "," was not ")
  text = text.replace(" wasn t "," was not ")
  text = text.replace(" cant "," can not ")
  text = text.replace(" can t "," can not ")
  text = text.replace(" couldnt "," could not ")
  text = text.replace(" couldn t "," could not ")
  text = text.replace(" wouldnt "," would not ")
  text = text.replace(" wouldn t "," would not ")
  text = text.replace(" dont "," do not ")
  text = text.replace(" don t "," do not ")
  text = text.replace(" didnt "," did not ")
  text = text.replace(" didn t "," did not ")
  text = text.replace(" let s "," let us ")
  text = text.replace(" i'm "," i am ")
  text = text.replace(" i've "," i have ")
  text = text.replace(" i'll "," i will ")
  text = text.replace(" i'd "," i had ")
  text = text.replace(" that's "," that is ")
  text = text.replace(" isn't "," is not ")
  text = text.replace(" it's "," it is ")
  text = text.replace(" she's "," she is ")
  text = text.replace(" he's "," he is ")
  text = text.replace(" u "," you ")
  text = text.replace(" ur "," your ")
  text = text.replace(" b4 "," before ")
  text = text.replace(" wasn't "," was not ")
  text = text.replace(" can't "," can not ")
  text = text.replace(" couldn't "," could not ")
  text = text.replace(" wouldn't "," would not ")
  text = text.replace(" don't "," do not ")
  text = text.replace(" didn't "," did not ")
  text = text.replace(" let's "," let us ")
  text = text.replace(" luv "," love ")
  text = text.replace(" true "," truth ")
  text = text.replace(" ppl "," people ")
  text = text.replace(" fb "," facebook ")
  text = text.replace(" b day "," birthday ")
  text = text.replace(" bday "," birthday ")
  if (len(text.strip())  == 0):
      text = ' ' #replace None with a single space
  return(text)

def removeHashtags(text):
  words = text.split()
  words = filter(lambda x:x[0]!='#', words)
  text = " ".join(words)
  if (len(text.strip())  == 0):
      text = ' ' #replace None with a single space
  return(text)

def removeMentions(text):
  words = text.split()
  words = filter(lambda x:x[0]!='@', words)
  text = " ".join(words)
  if (len(text.strip())  == 0):
      text = ' ' #replace None with a single space
  return(text)

def removeHttpWeb(text):
  words = text.split()
  words = filter(lambda x:x[0]!='&', words)
  words = filter(lambda x:x[0:4]!='http', words)
  text = " ".join(words)
  if (len(text.strip())  == 0):
      text = ' ' #replace None with a single space
  return(text)


In [12]:
# CLEANUP FACTORY
def cleanup(text):
  #Scenario1: 
  text = removeSpecialChar(text)
  text = remove1stMent(text)
  text = removeHttpWeb(text)
  text = cleanColloquials(text)

  #Scenario2: remove1stMent, removeHttpWeb, removeSpecialChar, cleanColloquials
  #Scenario3: remove1stMent, removeHttpWeb, removeHashtags, removeSpecialChar, cleanColloquials
  return(text)

In [13]:
#df['CleanTweet'] = df['OrigTweet'].apply(remove1stMentNoSpecChar)
#remove1stMentNoSpecChar('@MarkSZaidEsq @tko From the evidence and more to co')
df['CleanTweet'] = df['OrigTweet'].apply(cleanup)

In [14]:
print(f"The dataset contains { df.Target.nunique() } unique categories")

The dataset contains 5 unique categories


In [15]:
# convert the tweets into lower case if uncased model.
#df['CleanTweet'] = df['CleanTweet'].apply(lambda x: str(x).lower())
# trim to 280 characters max
df['CleanTweet'] = df['CleanTweet'].str.slice(0,279)
# calculating the length of tweet
df['CleanTweet_len'] = df['CleanTweet'].apply(lambda x: len(str(x).split()))
# Remove tweets with less than 10 words
df = df.query('CleanTweet_len > 9')
len(df)

101245

In [16]:
df['Label'].unique()

array(['Politics', 'Entertainment', 'Business', 'Sports', 'Technology'],
      dtype=object)

In [17]:
df['Target'].unique()

array([10,  6,  3, 11, 12])

In [18]:
#For the initial Ground Truth:
df.loc[(df.Target == 10), 'Target'] = 0
df.loc[(df.Target == 6), 'Target'] = 1
df.loc[(df.Target == 3), 'Target'] = 2
df.loc[(df.Target == 11), 'Target'] = 3
df.loc[(df.Target == 12), 'Target'] = 4


In [19]:
# set index to TID (used later in the Training/Test Series)
df.index = df['TID']
df.index.rename('gt_idx', inplace=True)

In [20]:
df.head()

,TID,OrigTweet,InReplyTo,Target,Label,CleanTweet,CleanTweet_len
gt_idx,,,,,,,
826262311560216578,826262311560216578,#coup has started. First of many steps. #rebel...,NaN,0,Politics,#coup has started. First of many steps. #rebel...,13
1193437298303438858,1193437298303438858,@MarkSZaidEsq @jody_prichard Funny you want to...,826262311560216578,0,Politics,@jody_prichard Funny you want to discriminate ...,47
1194280882540036098,1194280882540036098,"@MarkSZaidEsq at THAT time, the only ""stepping...",826262311560216578,0,Politics,"at THAT time, the only ""stepping over the line...",54
1194635634960478208,1194635634960478208,@MarkSZaidEsq It's not a coup. It's a Constit...,826262311560216578,0,Politics,It's not a coup. It's a Constitutional procedu...,47
1194636052096413696,1194636052096413696,"@MarkSZaidEsq BTW, I've read your client HOSTE...",826262311560216578,0,Politics,"BTW, I've read your client HOSTED (and was in)...",45


In [21]:
# BUILD THE MODEL
no_of_classes = 5
# seed for environmental stability 
# tf.random.set_seed(0) 

In [22]:
#LSTM tokenizer setup
emb_dim = 128
n_most_common_words = 80000
max_len = 99
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['CleanTweet'].values)
sequences = tokenizer.texts_to_sequences(df['CleanTweet'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

Found 74960 unique tokens.


In [23]:
#build the model on tpu
with strategy.scope():
  model = Sequential()
  model.add(Embedding(n_most_common_words, emb_dim, input_length=99))
  model.add(SpatialDropout1D(0.3))
  #model.add(Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3))) # for BiDirectional LSTM
  model.add(Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))) # for BiDirectional LSTM
  model.add(SeqWeightedAttention()) # add attention layer
  model.add(Dense(no_of_classes, activation='softmax'))
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())
# Save the initial Neural Net weights (to use for resetting the model in between cross validation runs)
init_weights = model.get_weights()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 99, 128)           10240000  
                                                                 
 spatial_dropout1d (SpatialD  (None, 99, 128)          0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 99, 128)          98816     
 l)                                                              
                                                                 
 seq_weighted_attention (Seq  (None, 128)              129       
 WeightedAttention)                                              
                                                                 
 dense (Dense)               (None, 5)                 645       
                                                        

/usr/local/lib/python3.10/dist-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


None


In [24]:
#X_train,X_test ,y_train,y_test = train_test_split(df['CleanTweet'], df['Target'], random_state = 1, stratify=df['Target'], test_size = 0.2)
# X_train = df['CleanTweet']
y = df['Target']

In [25]:
runTweak = "Mini cleanup, remove 1st mention, finetune LSTM"

In [26]:
# TRAIN THE MODEL

In [27]:
# run_functions_eagerly command executed to prevent"ValueError: Creating variables on a non-first call to a function decorated with tf.function" error
# Resolution from: https://stackoverflow.com/questions/58352326/running-the-tensorflow-2-0-code-gives-valueerror-tf-function-decorated-functio
# tf.config.run_functions_eagerly(True)

In [28]:
runID = dt.today().strftime('%Y%m%d%H%M')
# 5-Fold Cross Validation
kf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
# kf = StratifiedKFold(n_splits=2, random_state=0, shuffle=True)
# kf = StratifiedKFold(n_splits=2)
k=1
df_tr_hist = pd.DataFrame(columns=['RunID','RunTweak','Model','Loss','Accuracy'])
df_val_result = pd.DataFrame(columns=['ValID','ValType','Model','Accuracy','Precision','Recall','F1'])
for tr_idx, val_idx in kf.split(X, y):
  print(f"\nTraining for Cross Val: {k} ")
  print(f"Indexes for tr {tr_idx} and val {val_idx}")

  # create training and validation datasets
  X_tr, X_val = X[tr_idx], X[val_idx]
  y_tr, y_val = y[tr_idx], y[val_idx]
  
  # Encode y_tr and y_val
  ytr_encoded = tf.keras.utils.to_categorical(y_tr, num_classes=no_of_classes,dtype = 'int32')
  yval_encoded = tf.keras.utils.to_categorical(y_val, num_classes=no_of_classes,dtype = 'int32')

  # Encode X_tr and X_val -- ALREADY ENCODED IN X

  #training for the epochs
  train_history = model.fit(
      x=X_tr, y=ytr_encoded,
      # steps_per_epoch = n_steps,
      batch_size=128,
      epochs=10,
      # validation_split=0.2,
      callbacks=[EarlyStopping(monitor='loss',patience=7, min_delta=0.0001)]
      )

  #store the training history log
  df_tr_hist.loc[len(df_tr_hist)] = [runID, runTweak, 'LSTM', train_history.history['loss'], train_history.history['accuracy']]

  #Making predictions for val
  preds = model.predict(X_val,verbose = 1)
  pred_classes = np.argmax(preds, axis = 1)
  
  #Mapping the encoded output to actual categories...
  # write result for decomposed SMR
  actual_category = np.argmax(yval_encoded, axis = 1) 
  acc = sklearn.metrics.accuracy_score(actual_category,pred_classes)
  print("Generating decomposed SMR performance metrics ..." )
  de_prec = precision_score(actual_category,pred_classes, average = 'weighted')
  de_recall = recall_score(actual_category,pred_classes, average = 'weighted')
  de_f1 = f1_score(actual_category,pred_classes, average = 'weighted')
  print(f"Decomposed SMR Acc: {acc*100}, Prec: {de_prec*100}, Recall: {de_recall*100}, F1 {de_f1*100}")
  df_val_result.loc[len(df_val_result)] = [k, 'Decomposed', 'LSTM', acc*100, de_prec*100, de_recall*100, de_f1*100]

  # p_r_f1 = precision_recall_fscore_support(actual_category, pred_classes, average = 'weighted')
  # print(f"Decomposed SMR Acc: {acc*100}, Prec: {p_r_f1[0]*100}, Recall: {p_r_f1[1]*100}, F1 {p_r_f1[2]*100}")
  # df_val_result.loc[len(df_val_result)] = [k, 'Decomposed', 'LSTM', acc*100, p_r_f1[0]*100, p_r_f1[1]*100, p_r_f1[2]*100]

  print("Generating recomposed SMR accuracy...")
  list_y_val_idx = y_val.index.tolist()
  result_df = pd.DataFrame({'idx':list_y_val_idx, 'actual_category':actual_category, 'predicted_category':pred_classes})
  df_sup_result = result_df.merge(df, left_on='idx', right_on='TID')
  df_sup_agg_result = df_sup_result.groupby(['InReplyTo','predicted_category']).size().reset_index(name='counts').sort_values(['InReplyTo','counts'], ascending=[True,False])
  df_rec = df[df.InReplyTo.isna()]

  # Recompose the SMR based on the count of aggregated supporting tweet predictions
  df_rec_result = pd.DataFrame(columns=['TID','Actual','Predicted'])
  for i in range(0,len(df_rec)):
    rec_TID = df_rec.iloc[i]['TID']
    rec_cat = int(df_rec.iloc[i]['Target'])
    df_temp = df_sup_agg_result[df_sup_agg_result.InReplyTo == rec_TID].sort_values('counts',ascending=False)
    if len(df_temp) > 0:
      rec_pred = int(df_temp[:1]['predicted_category'].values)
      df_rec_result.loc[len(df_rec_result)] = [rec_TID,rec_cat,rec_pred]

  # write result for recomposed SMR
  rec_actual = df_rec_result['Actual'].to_numpy().astype(int)
  rec_pred = df_rec_result['Predicted'].to_numpy().astype(int)
  rec_acc = sklearn.metrics.accuracy_score(rec_actual, rec_pred)
  rec_prec = precision_score(rec_actual, rec_pred, average = 'weighted')
  rec_recall = recall_score(rec_actual, rec_pred, average = 'weighted')
  rec_f1 = f1_score(rec_actual, rec_pred, average = 'weighted')
  print(f"Recomposed SMR Acc: {rec_acc*100}, Prec: {rec_prec*100}, Recall: {rec_recall*100}, F1 {rec_f1*100}")
  df_val_result.loc[len(df_val_result)] = [k, 'Recomposed', 'LSTM', rec_acc*100, rec_prec*100, rec_recall*100, rec_f1*100]

  # rec_p_r_f1 = precision_recall_fscore_support(rec_actual, rec_pred, average = 'weighted')
  # print(f"Recomposed SMR Acc: {rec_acc*100}, Prec: {rec_p_r_f1[0]*100}, Recall: {rec_p_r_f1[1]*100}, F1 {rec_p_r_f1[2]*100}")
  # df_val_result.loc[len(df_val_result)] = [k, 'Recomposed', 'LSTM', rec_acc*100, rec_p_r_f1[0]*100, rec_p_r_f1[1]*100, rec_p_r_f1[2]*100]

  # Generate confusion matrix
  rec_cm = confusion_matrix(rec_actual, rec_pred)
  df_rec_cm = pd.DataFrame(rec_cm)
  df_rec_cm.to_csv("/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/results/LSTM_GTr_CM_" + str(k) + " " + runID +".csv")

  # Reset the model by restoring the initital model weights to prevent overtraining
  model.set_weights(init_weights)
  k=k+1

# Save the training history result
df_tr_hist.to_csv("/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/results/LSTM_GTr_TrainHist_" + runID +".csv")

# Save the cross validation result
df_val_result = df_val_result.round(decimals=2)
df_val_result.to_csv("/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/results/LSTM_GTr_CV_" + runID +".csv")

# Save RunID,Model,Accuracy,Precision,Recall,F1 for the Recomposed SMR
df_re_val_result = df_val_result[df_val_result.ValType=='Recomposed']
no_of_runs = len(df_re_val_result)
df_GTxM_Clf_CV_results = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/results/GTxM_Clf_CV_results.csv")
avg_Accuracy = np.sum(df_re_val_result['Accuracy']) / no_of_runs
avg_Precision = np.sum(df_re_val_result['Precision']) / no_of_runs
avg_Recall = np.sum(df_re_val_result['Recall']) / no_of_runs
avg_F1 = np.sum(df_re_val_result['F1']) / no_of_runs
df_GTxM_Clf_CV_results.loc[len(df_GTxM_Clf_CV_results)] = [runID, 'LSTM', avg_Accuracy, avg_Precision, avg_Recall, avg_F1]
df_GTxM_Clf_CV_results.to_csv("/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/results/GTxM_Clf_CV_results.csv", index=False)


Training for Cross Val: 1 
Indexes for tr [     0      1      2 ... 101241 101242 101243] and val [     5      8     16 ... 101218 101220 101244]
Epoch 1/10
633/633 [==============================] - 53s 57ms/step - loss: 1.4242 - accuracy: 0.4931
Epoch 2/10
633/633 [==============================] - 29s 46ms/step - loss: 1.3031 - accuracy: 0.4965
Epoch 3/10
633/633 [==============================] - 29s 45ms/step - loss: 1.2282 - accuracy: 0.5311
Epoch 4/10
633/633 [==============================] - 27s 43ms/step - loss: 1.0415 - accuracy: 0.6324
Epoch 5/10
633/633 [==============================] - 27s 43ms/step - loss: 0.9916 - accuracy: 0.6525
Epoch 6/10
633/633 [==============================] - 28s 44ms/step - loss: 0.9617 - accuracy: 0.6641
Epoch 7/10
633/633 [==============================] - 27s 43ms/step - loss: 0.9377 - accuracy: 0.6728
Epoch 8/10
633/633 [==============================] - 27s 43ms/step - loss: 0.9177 - accuracy: 0.6789
Epoch 9/10
633/633 [=================

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recomposed SMR Acc: 72.86821705426357, Prec: 58.26271455689576, Recall: 72.86821705426357, F1 64.23403916545351

Training for Cross Val: 2 
Indexes for tr [     0      2      5 ... 101242 101243 101244] and val [     1      3      4 ... 101230 101233 101234]
Epoch 1/10
633/633 [==============================] - 28s 44ms/step - loss: 1.4635 - accuracy: 0.4917
Epoch 2/10
633/633 [==============================] - 28s 44ms/step - loss: 1.3076 - accuracy: 0.4965
Epoch 3/10
633/633 [==============================] - 28s 43ms/step - loss: 1.3023 - accuracy: 0.4965
Epoch 4/10
633/633 [==============================] - 28s 45ms/step - loss: 1.2997 - accuracy: 0.4965
Epoch 5/10
633/633 [==============================] - 28s 43ms/step - loss: 1.2963 - accuracy: 0.4965
Epoch 6/10
633/633 [==============================] - 28s 44ms/step - loss: 1.2899 - accuracy: 0.4965
Epoch 7/10
633/633 [==============================] - 27s 43ms/step - loss: 1.2806 - accuracy: 0.4976
Epoch 8/10
633/633 [=======

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recomposed SMR Acc: 61.65605095541401, Prec: 54.71452742369277, Recall: 61.65605095541401, F1 55.13832027844767

Training for Cross Val: 3 
Indexes for tr [     0      1      3 ... 101242 101243 101244] and val [     2      7     22 ... 101225 101236 101240]
Epoch 1/10
633/633 [==============================] - 28s 44ms/step - loss: 1.3943 - accuracy: 0.4930
Epoch 2/10
633/633 [==============================] - 27s 43ms/step - loss: 1.3043 - accuracy: 0.4965
Epoch 3/10
633/633 [==============================] - 28s 44ms/step - loss: 1.2993 - accuracy: 0.4965
Epoch 4/10
633/633 [==============================] - 28s 43ms/step - loss: 1.2829 - accuracy: 0.4969
Epoch 5/10
633/633 [==============================] - 27s 43ms/step - loss: 1.1909 - accuracy: 0.5368
Epoch 6/10
633/633 [==============================] - 27s 43ms/step - loss: 1.1298 - accuracy: 0.5736
Epoch 7/10
633/633 [==============================] - 27s 43ms/step - loss: 1.0939 - accuracy: 0.5976
Epoch 8/10
633/633 [=======

...

Recomposed SMR Acc: 70.03891050583657, Prec: 58.23284071339009, Recall: 70.03891050583657, F1 62.27118167537391

Training for Cross Val: 4 
Indexes for tr [     0      1      2 ... 101242 101243 101244] and val [    14     15     19 ... 101231 101238 101239]
Epoch 1/10
633/633 [==============================] - 28s 44ms/step - loss: 1.4004 - accuracy: 0.4906
Epoch 2/10
633/633 [==============================] - 28s 44ms/step - loss: 1.3026 - accuracy: 0.4965
Epoch 3/10
633/633 [==============================] - 28s 44ms/step - loss: 1.2984 - accuracy: 0.4965
Epoch 4/10
633/633 [==============================] - 28s 44ms/step - loss: 1.2915 - accuracy: 0.4965
Epoch 5/10
633/633 [==============================] - 28s 45ms/step - loss: 1.2846 - accuracy: 0.4965
Epoch 6/10
633/633 [==============================] - 27s 43ms/step - loss: 1.2795 - accuracy: 0.4979
Epoch 7/10
633/633 [==============================] - 28s 44ms/step - loss: 1.2738 - accuracy: 0.5003
Epoch 8/10
633/633 [=======

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recomposed SMR Acc: 63.0012936610608, Prec: 54.982376259439405, Recall: 63.0012936610608, F1 56.11726853633648

Training for Cross Val: 5 
Indexes for tr [     1      2      3 ... 101239 101240 101244] and val [     0      6      9 ... 101241 101242 101243]
Epoch 1/10
633/633 [==============================] - 28s 44ms/step - loss: 1.4002 - accuracy: 0.4931
Epoch 2/10
633/633 [==============================] - 28s 44ms/step - loss: 1.3050 - accuracy: 0.4965
Epoch 3/10
633/633 [==============================] - 28s 44ms/step - loss: 1.3002 - accuracy: 0.4965
Epoch 4/10
633/633 [==============================] - 28s 44ms/step - loss: 1.2941 - accuracy: 0.4965
Epoch 5/10
633/633 [==============================] - 28s 44ms/step - loss: 1.2400 - accuracy: 0.5097
Epoch 6/10
633/633 [==============================] - 28s 43ms/step - loss: 1.1446 - accuracy: 0.5682
Epoch 7/10
633/633 [==============================] - 28s 44ms/step - loss: 1.1037 - accuracy: 0.5881
Epoch 8/10
633/633 [========

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recomposed SMR Acc: 67.31770833333334, Prec: 58.45250808001398, Recall: 67.31770833333334, F1 60.37741047282087


In [29]:
len(df_rec)

806